<a href="https://colab.research.google.com/github/DeerBay/Deep-Learning-Julia/blob/main/sequential_reward_shaping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Space Invaders with Deep Q-Learning and Reward Shaping
This notebook demonstrates a reinforcement learning project to train an agent to play Space Invaders using the Deep Q-Learning (DQN) algorithm. To enhance learning efficiency, reward shaping techniques are applied to guide the agent's behavior toward desired outcomes.

## Setup
The first few cells handle the installation of required libraries and the setup of Google Drive for saving models and logs. Ensure all dependencies are installed and paths are correctly set.

## Reward Shaping
The `RewardShaper` class adds domain knowledge to the reward signal. For instance, it:
- Encourages score improvements with small bonus rewards.
- Penalizes losing lives or inactivity.
- Incentivizes movement and shooting with balanced adjustments.

These modifications help guide the agent toward optimal behavior.

I've tried three different reward shapings here to examine wether it is possible to train the model for shorter time with effecient result.

## Deep Q-Learning Agent
The DQN agent uses a convolutional neural network to approximate the Q-value function. Key components include:
- **Network Architecture**: Three convolutional layers followed by a fully connected layer.
- **Action Selection**: Epsilon-greedy policy balances exploration and exploitation.
- **Target Network**: A separate target network helps stabilize Q-value updates.
- **Training**: The agent minimizes the Huber loss between predicted and target Q-values.

## Training Process
The main training loop involves resetting the environment, interacting with it using the agent's policy, and updating the Q-network using experience replay. Metrics such as shaped rewards, original rewards, and epsilon values are logged for analysis.

## Visualization
After each try I've generated a plot showing the training progress:
- **Average Reward**: Monitors the agent's performance over episodes.
- **Epsilon**: Tracks the exploration rate as it decays over time.

These insights help evaluate the effectiveness of the training and reward shaping techniques.

In [ ]:
!pip install 'gymnasium[atari, other]' gymnasium tensorflow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Try 1

Looking at the reward shaping implementation in the code, the key aspects of preferred rewards are:

Score-Based Rewards:


Improvements in game score are rewarded, but with a moderate multiplier (0.05)
This is intentionally reduced from 0.1 to avoid over-emphasizing score chasing


Survival-Based Rewards:


Loss of lives is penalized (-0.5)
This creates a strong incentive for survival behavior


Movement-Based Rewards:


Small reward (0.005) for movement to encourage exploration
Penalties (-0.05) for staying still too long (after 15 frames)
This balance prevents both camping and excessive movement


Shooting Behavior:


Moderate reward (0.02) for shooting actions
Small penalty (-0.02) for not shooting for extended periods (>30 frames)
The timing and values are carefully balanced to encourage periodic but not constant shooting


Reward Bounds:


A floor of -0.5 is implemented to prevent extreme negative rewards
This helps maintain stable learning by avoiding reward collapse

The overall philosophy appears to prefer balanced, moderate rewards that:

Don't overemphasize any single behavior
Encourage a mix of survival, scoring, and tactical play
Avoid extreme values that could destabilize learning
Create natural-feeling gameplay without exploitable patterns

Would you like me to elaborate on any of these aspects or explain why certain reward values were chosen over others?

In [ ]:
import tensorflow as tf
from keras import layers
import gymnasium as gym
from gymnasium.wrappers import FrameStackObservation, AtariPreprocessing
import numpy as np
import os
import ale_py

class ReplayBuffer:
    def __init__(self, max_size, state_shape, num_actions):
        self.max_size = max_size
        self.states = np.zeros((max_size, *state_shape), dtype=np.float32)
        self.actions = np.zeros(max_size, dtype=np.int32)
        self.rewards = np.zeros(max_size, dtype=np.float32)
        self.next_states = np.zeros((max_size, *state_shape), dtype=np.float32)
        self.dones = np.zeros(max_size, dtype=np.float32)
        self.index = 0
        self.is_full = False

    def add(self, state, action, reward, next_state, done):
        idx = self.index % self.max_size
        self.states[idx] = state
        self.actions[idx] = action
        self.rewards[idx] = reward
        self.next_states[idx] = next_state
        self.dones[idx] = done
        self.index += 1
        if self.index >= self.max_size:
            self.is_full = True

    def sample(self, batch_size):
        max_index = self.max_size if self.is_full else self.index
        indices = np.random.choice(max_index, batch_size, replace=False)
        return (
            self.states[indices],
            self.actions[indices],
            self.rewards[indices],
            self.next_states[indices],
            self.dones[indices]
        )

    def __len__(self):
        return self.max_size if self.is_full else self.index

class RewardShaper:
    def __init__(self):
        self.previous_score = 0
        self.previous_lives = 3
        self.steps_since_last_shot = 0
        self.last_known_position = None
        self.consecutive_no_movement = 0

    def calculate_shaped_reward(self, info, reward, action, position):
        shaped_reward = reward  # Start with original reward

        # Track score improvements - reduced multiplier
        current_score = info.get('score', 0)
        if current_score > self.previous_score:
            score_improvement = current_score - self.previous_score
            shaped_reward += 0.05 * score_improvement  # Reduced from 0.1 to 0.05
            self.previous_score = current_score

        # Penalize death
        current_lives = info.get('lives', 3)
        if current_lives < self.previous_lives:
            shaped_reward -= 0.5
            self.previous_lives = current_lives

        # Movement-based shaping
        if self.last_known_position is not None:
            movement = abs(position - self.last_known_position)
            if movement < 0.005:
                self.consecutive_no_movement += 1
                if self.consecutive_no_movement > 15:
                    shaped_reward -= 0.05
            else:
                self.consecutive_no_movement = 0
                # Smaller movement reward
                shaped_reward += 0.005

        self.last_known_position = position

        # Shooting behavior shaping - more balanced
        if action == 1:  # Shoot action
            self.steps_since_last_shot = 0
            shaped_reward += 0.02  # Reduced from 0.05 to 0.02
        else:
            self.steps_since_last_shot += 1
            if self.steps_since_last_shot > 30:  # Increased from 20 to 30
                shaped_reward -= 0.02  # Reduced from 0.05 to 0.02

        # Ensure reward doesn't go too negative
        return max(-0.5, shaped_reward)  # Add a floor to prevent extreme negative rewards

class DQNAgent:
    def __init__(self, state_shape, num_actions, learning_rate=0.00025):
        self.state_shape = state_shape
        self.num_actions = num_actions
        self.model = self.create_q_model(state_shape, num_actions)
        self.target_model = self.create_q_model(state_shape, num_actions)
        self.target_model.set_weights(self.model.get_weights())
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0)
        self.replay_buffer = ReplayBuffer(max_size=100000, state_shape=state_shape, num_actions=num_actions)
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_max = 1.0
        self.epsilon_random_frames = 10000  # Initial random frames
        self.epsilon_greedy_frames = 100000.0  # Total frames over which to decay epsilon

    def create_q_model(self, input_shape, num_actions):
        return tf.keras.Sequential([
            layers.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=8, strides=4, activation="relu", kernel_initializer='he_uniform'),
            layers.Conv2D(64, kernel_size=4, strides=2, activation="relu", kernel_initializer='he_uniform'),
            layers.Conv2D(64, kernel_size=3, strides=1, activation="relu", kernel_initializer='he_uniform'),
            layers.Flatten(),
            layers.Dense(512, activation="relu", kernel_initializer='he_uniform'),
            layers.Dense(num_actions, activation="linear")
        ])

    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(self.num_actions)
        state_tensor = tf.convert_to_tensor(state[np.newaxis, ...], dtype=tf.float32)
        q_values = self.model(state_tensor, training=False)
        return tf.argmax(q_values[0]).numpy()

    @tf.function
    def train_step(self, states, actions, rewards, next_states, dones):
        future_rewards = self.target_model(next_states, training=False)
        target_q_values = rewards + self.gamma * tf.reduce_max(future_rewards, axis=1) * (1 - dones)
        with tf.GradientTape() as tape:
            q_values = self.model(states, training=True)
            q_action = tf.reduce_sum(tf.one_hot(actions, self.num_actions) * q_values, axis=1)
            loss = tf.keras.losses.Huber()(target_q_values, q_action)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

def get_agent_position(state):
    """Extract the agent's position from the state"""
    bottom_slice = state[-10:, :, -1]  # Look at the bottom portion of the last frame
    agent_x = np.mean(np.where(bottom_slice > 0.5)[1]) if np.any(bottom_slice > 0.5) else None
    if agent_x is not None:
        return agent_x / state.shape[1]  # Normalize position
    return 0.5  # Return center if agent not found

def main():
    # Paths for saving
    model_dir = "/content/drive/MyDrive/SpaceInvaders/models"
    plot_dir = "/content/drive/MyDrive/SpaceInvaders/plots"
    log_dir = "/content/drive/MyDrive/SpaceInvaders/logs"
    log_file_path = os.path.join(log_dir, "training_log.txt")

    # Create all necessary directories
    for directory in [model_dir, plot_dir, log_dir]:
        os.makedirs(directory, exist_ok=True)

    # Environment setup
    gym.register_envs(ale_py)
    env = gym.make("SpaceInvadersNoFrameskip-v4", render_mode="rgb_array")
    env = AtariPreprocessing(env, frame_skip=4, grayscale_obs=True, scale_obs=True)
    env = FrameStackObservation(env, stack_size=4)

    # Agent and reward shaper setup
    state_shape = (84, 84, 4)
    num_actions = env.action_space.n
    agent = DQNAgent(state_shape, num_actions)
    reward_shaper = RewardShaper()

    # Training parameters
    max_episodes = 750  # Reduced from 2000
    max_steps_per_episode = 3000
    update_target_every = 1000
    log_every = 1
    episode_rewards = []
    frame_count = 0
    best_avg_reward = -float('inf')

    with open(log_file_path, "w") as f:
        f.write("Episode,Avg_Reward,Epsilon,Original_Reward,Shaped_Reward\n")

    # Training loop
    for episode in range(max_episodes):
        state, info = env.reset()
        state = np.array(state, dtype=np.float32) / 255.0
        state = np.transpose(state, (1, 2, 0))
        episode_reward = 0
        episode_original_reward = 0
        episode_steps = 0

        for step in range(max_steps_per_episode):
            frame_count += 1
            episode_steps += 1

            # Choose action
            if frame_count < agent.epsilon_random_frames:
                action = np.random.randint(num_actions)
            else:
                action = agent.get_action(state)

            # Step environment
            next_state, reward, done, _, info = env.step(action)
            next_state = np.array(next_state, dtype=np.float32) / 255.0
            next_state = np.transpose(next_state, (1, 2, 0))

            # Shape the reward
            agent_position = get_agent_position(state)
            shaped_reward = reward_shaper.calculate_shaped_reward(info, reward, action, agent_position)

            # Store original reward for logging
            episode_original_reward += reward
            episode_reward += shaped_reward

            # Store in replay buffer
            agent.replay_buffer.add(state, action, shaped_reward, next_state, done)
            state = next_state

            # Train
            if len(agent.replay_buffer) >= 32:
                states, actions, rewards, next_states, dones = agent.replay_buffer.sample(32)
                agent.train_step(states, actions, rewards, next_states, dones)

            # Update target model
            if frame_count % update_target_every == 0:
                agent.update_target_model()

            if done:
                break

        # Episode-based epsilon decay
        if frame_count >= agent.epsilon_random_frames:
            decay_per_episode = (agent.epsilon_max - agent.epsilon_min) / (max_episodes * 0.3)  # Decay over first 30% of episodes
            agent.epsilon = max(agent.epsilon_min, agent.epsilon - decay_per_episode)

        episode_rewards.append(episode_reward)
        avg_reward = np.mean(episode_rewards[-log_every:])

        # Log both original and shaped rewards
        with open(log_file_path, "a") as f:
            f.write(f"{episode + 1},{avg_reward:.2f},{agent.epsilon:.4f},{episode_original_reward:.2f},{episode_reward:.2f}\n")

        print(f"Episode {episode + 1}, Avg Reward: {avg_reward:.2f}, Original Reward: {episode_original_reward:.2f}, "
              f"Shaped Reward: {episode_reward:.2f}, Epsilon: {agent.epsilon:.4f}, Frame Count: {frame_count}, "
              f"Steps: {episode_steps}")

        # Save model conditionally
        if (episode + 1) % 100 == 0 or avg_reward > best_avg_reward:
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
                print(f"New best average reward: {best_avg_reward:.2f}. Saving best model.")
            agent.model.save(f"{model_dir}/episode{episode + 1}_space_invaders_model.keras")

    # Save final model
    agent.model.save(f"{model_dir}/final_space_invaders_model.keras")

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd
import matplotlib as plt

filepath ='/content/drive/MyDrive/SpaceInvaders/logs/training_log.txt'

df = pd.read_csv(filepath, sep=',')

# Directory to save the plot
save_path = '/content/drive/MyDrive/SpaceInvaders/plots/training_log_plot_1.png'

import matplotlib.pyplot as plt

# Create a figure with two y-axes
fig, ax1 = plt.subplots(figsize=(15, 7))

# Plot Average Reward on the primary y-axis
color1 = 'blue'
ax1.set_xlabel('Episode')
ax1.set_ylabel('Average Reward', color=color1)
ax1.plot(df['Episode'], df['Avg_Reward'], color=color1, alpha=0.7, label='Average Reward')
ax1.tick_params(axis='y', labelcolor=color1)

# Create a secondary y-axis for Epsilon
ax2 = ax1.twinx()
color2 = 'red'
ax2.set_ylabel('Epsilon', color=color2)
ax2.plot(df['Episode'], df['Epsilon'], color=color2, alpha=0.7, label='Epsilon')
ax2.tick_params(axis='y', labelcolor=color2)

# Title and layout
plt.title('Reinforcement Learning Training Log: Reward and Exploration Rate', fontsize=16)
fig.tight_layout()

# Add a legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Grid for better readability
ax1.grid(True, linestyle='--', alpha=0.7)

# Save the plot
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"Plot saved at: {save_path}")

# Optional: Calculate and print some statistics
print(f"Total Episodes: {len(df)}")
print(f"\nReward Statistics:")
print(f"Minimum Reward: {df['Avg_Reward'].min():.2f}")
print(f"Maximum Reward: {df['Avg_Reward'].max():.2f}")
print(f"Mean Reward: {df['Avg_Reward'].mean():.2f}")
print(f"Median Reward: {df['Avg_Reward'].median():.2f}")

print(f"\nEpsilon Statistics:")
print(f"Initial Epsilon: {df['Epsilon'].iloc[0]:.4f}")
print(f"Final Epsilon: {df['Epsilon'].iloc[-1]:.4f}")

# Show the plot (optional, depends on your environment)
plt.show()

## Outcome Try 1

1. Epsilon Decay (Red Line):
- Starts at 1.0 (100% random actions)
- Steadily decreases until around episode 300
- Stabilizes at the minimum value (0.1) for the remainder of training
- This indicates a proper transition from exploration to exploitation

2. Average Reward Performance (Blue Line):
- Initial performance is relatively low and unstable (0-200 episodes)
- Shows increasing variance and higher peaks as training progresses
- Notable spikes reaching ~600-1000 rewards occur periodically
- The baseline performance seems to stabilize around 200-400 rewards

3. Learning Characteristics:
- High variance in performance even after epsilon stabilization
- No clear consistent upward trend in average reward
- Periodic high-performance episodes suggest the agent learns useful strategies but struggles with consistency

4. Potential Issues:
- The high variance in rewards might indicate:
  - Sensitivity to initial conditions in episodes
  - Possible instability in the reward shaping mechanism
  - Challenge in maintaining consistent performance

This outcome suggests that while the agent can achieve high scores occasionally, the reward shaping approach might need refinement to:
- Reduce performance variance
- Improve consistency in learned behaviors
- Better stabilize the baseline performance

## Try 2

Looking at the code for this modified version, there are several changes that align with addressing the high variance issues we observed:

1. Improved Epsilon Decay Strategy:
- Changed from 30% to 70% of episodes for decay period (`max_episodes * 0.7`)
- Lowered epsilon_min from 0.1 to 0.01
- Increased initial random frames from 10,000 to 50,000
- This should provide more stable exploration and better policy learning

2. More Balanced Reward Shaping:
- Death penalty reduced but still significant (-0.5)
- Movement thresholds made more lenient (0.005 vs 0.01)
- Shooting behavior rewards balanced (0.02) with longer tolerance (30 frames vs 20)
- Floor added to prevent extreme negative rewards (max(-0.5, shaped_reward))

3. Training Parameters:
- Increased max_episodes from 750 to 1000
- Kept max_steps_per_episode at 3000
- Maintained update_target_every at 1000

These changes should help by:
- Allowing more time for exploration and policy refinement
- Reducing reward volatility while maintaining meaningful signals
- Creating a smoother learning curve with less extreme variations

I believe this could help address the high variance issues we saw in the previous training results. Would you like me to explain any specific aspect of these modifications in more detail?

In [ ]:

import tensorflow as tf
from keras import layers
import gymnasium as gym
from gymnasium.wrappers import FrameStackObservation, AtariPreprocessing
import numpy as np
import os
import ale_py
import matplotlib as plt

class ReplayBuffer:
    def __init__(self, max_size, state_shape, num_actions):
        self.max_size = max_size
        self.states = np.zeros((max_size, *state_shape), dtype=np.float32)
        self.actions = np.zeros(max_size, dtype=np.int32)
        self.rewards = np.zeros(max_size, dtype=np.float32)
        self.next_states = np.zeros((max_size, *state_shape), dtype=np.float32)
        self.dones = np.zeros(max_size, dtype=np.float32)
        self.index = 0
        self.is_full = False

    def add(self, state, action, reward, next_state, done):
        idx = self.index % self.max_size
        self.states[idx] = state
        self.actions[idx] = action
        self.rewards[idx] = reward
        self.next_states[idx] = next_state
        self.dones[idx] = done
        self.index += 1
        if self.index >= self.max_size:
            self.is_full = True

    def sample(self, batch_size):
        max_index = self.max_size if self.is_full else self.index
        indices = np.random.choice(max_index, batch_size, replace=False)
        return (
            self.states[indices],
            self.actions[indices],
            self.rewards[indices],
            self.next_states[indices],
            self.dones[indices]
        )

    def __len__(self):
        return self.max_size if self.is_full else self.index

class RewardShaper:
    def __init__(self):
        self.previous_score = 0
        self.previous_lives = 3
        self.steps_since_last_shot = 0
        self.last_known_position = None
        self.consecutive_no_movement = 0

    def calculate_shaped_reward(self, info, reward, action, position):
        shaped_reward = reward  # Start with original reward

        # Track score improvements - reduced multiplier
        current_score = info.get('score', 0)
        if current_score > self.previous_score:
            score_improvement = current_score - self.previous_score
            shaped_reward += 0.05 * score_improvement  # Reduced from 0.1 to 0.05
            self.previous_score = current_score

        # Penalize death - made less punitive
        current_lives = info.get('lives', 3)
        if current_lives < self.previous_lives:
            shaped_reward -= 0.5  # Reduced from 1.0 to 0.5
            self.previous_lives = current_lives

        # Movement-based shaping - more lenient thresholds
        if self.last_known_position is not None:
            movement = abs(position - self.last_known_position)
            if movement < 0.005:  # Reduced threshold from 0.01 to 0.005
                self.consecutive_no_movement += 1
                if self.consecutive_no_movement > 15:  # Increased from 10 to 15
                    shaped_reward -= 0.05  # Reduced penalty from 0.1 to 0.05
            else:
                self.consecutive_no_movement = 0
                # Smaller movement reward
                shaped_reward += 0.005  # Reduced from 0.01 to 0.005

        self.last_known_position = position

        # Shooting behavior shaping - more balanced
        if action == 1:  # Shoot action
            self.steps_since_last_shot = 0
            shaped_reward += 0.02  # Reduced from 0.05 to 0.02
        else:
            self.steps_since_last_shot += 1
            if self.steps_since_last_shot > 30:  # Increased from 20 to 30
                shaped_reward -= 0.02  # Reduced from 0.05 to 0.02

        # Ensure reward doesn't go too negative
        return max(-0.5, shaped_reward)  # Add a floor to prevent extreme negative rewards


class DQNAgent:
    def __init__(self, state_shape, num_actions, learning_rate=0.00025):
        self.state_shape = state_shape
        self.num_actions = num_actions
        self.model = self.create_q_model(state_shape, num_actions)
        self.target_model = self.create_q_model(state_shape, num_actions)
        self.target_model.set_weights(self.model.get_weights())
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0)
        self.replay_buffer = ReplayBuffer(max_size=100000, state_shape=state_shape, num_actions=num_actions)
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_max = 1.0
        self.epsilon_random_frames = 50000  # Initial random frames
        self.epsilon_greedy_frames = 500000.0  # Total frames over which to decay epsilon

    def create_q_model(self, input_shape, num_actions):
        return tf.keras.Sequential([
            layers.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=8, strides=4, activation="relu", kernel_initializer='he_uniform'),
            layers.Conv2D(64, kernel_size=4, strides=2, activation="relu", kernel_initializer='he_uniform'),
            layers.Conv2D(64, kernel_size=3, strides=1, activation="relu", kernel_initializer='he_uniform'),
            layers.Flatten(),
            layers.Dense(512, activation="relu", kernel_initializer='he_uniform'),
            layers.Dense(num_actions, activation="linear")
        ])

    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(self.num_actions)
        state_tensor = tf.convert_to_tensor(state[np.newaxis, ...], dtype=tf.float32)
        q_values = self.model(state_tensor, training=False)
        return tf.argmax(q_values[0]).numpy()

    @tf.function
    def train_step(self, states, actions, rewards, next_states, dones):
        future_rewards = self.target_model(next_states, training=False)
        target_q_values = rewards + self.gamma * tf.reduce_max(future_rewards, axis=1) * (1 - dones)
        with tf.GradientTape() as tape:
            q_values = self.model(states, training=True)
            q_action = tf.reduce_sum(tf.one_hot(actions, self.num_actions) * q_values, axis=1)
            loss = tf.keras.losses.Huber()(target_q_values, q_action)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

def get_agent_position(state):
    """Extract the agent's position from the state"""
    bottom_slice = state[-10:, :, -1]  # Look at the bottom portion of the last frame
    agent_x = np.mean(np.where(bottom_slice > 0.5)[1]) if np.any(bottom_slice > 0.5) else None
    if agent_x is not None:
        return agent_x / state.shape[1]  # Normalize position
    return 0.5  # Return center if agent not found

def main():
    # Paths for saving
    model_dir = "/content/drive/MyDrive/SpaceInvaders/models_2"
    plot_dir = "/content/drive/MyDrive/SpaceInvaders/plots_2"
    log_dir = "/content/drive/MyDrive/SpaceInvaders/logs_2"
    log_file_path = os.path.join(log_dir, "training_log_2.txt")

    # Create all necessary directories
    for directory in [model_dir, plot_dir, log_dir]:
        os.makedirs(directory, exist_ok=True)

    # Environment setup
    gym.register_envs(ale_py)
    env = gym.make("SpaceInvadersNoFrameskip-v4", render_mode="rgb_array")
    env = AtariPreprocessing(env, frame_skip=4, grayscale_obs=True, scale_obs=True)
    env = FrameStackObservation(env, stack_size=4)

    # Agent and reward shaper setup
    state_shape = (84, 84, 4)
    num_actions = env.action_space.n
    agent = DQNAgent(state_shape, num_actions)
    reward_shaper = RewardShaper()

    # Training parameters
    max_episodes = 1000  # Reduced from 2000
    max_steps_per_episode = 3000
    update_target_every = 1000
    log_every = 1
    episode_rewards = []
    frame_count = 0
    best_avg_reward = -float('inf')

    with open(log_file_path, "w") as f:
        f.write("Episode,Avg_Reward,Epsilon,Original_Reward,Shaped_Reward\n")

    # Training loop
    for episode in range(max_episodes):
        state, info = env.reset()
        state = np.array(state, dtype=np.float32) / 255.0
        state = np.transpose(state, (1, 2, 0))
        episode_reward = 0
        episode_original_reward = 0
        episode_steps = 0

        for step in range(max_steps_per_episode):
            frame_count += 1
            episode_steps += 1

            # Choose action
            if frame_count < agent.epsilon_random_frames:
                action = np.random.randint(num_actions)
            else:
                action = agent.get_action(state)

            # Step environment
            next_state, reward, done, _, info = env.step(action)
            next_state = np.array(next_state, dtype=np.float32) / 255.0
            next_state = np.transpose(next_state, (1, 2, 0))

            # Shape the reward
            agent_position = get_agent_position(state)
            shaped_reward = reward_shaper.calculate_shaped_reward(info, reward, action, agent_position)

            # Store original reward for logging
            episode_original_reward += reward
            episode_reward += shaped_reward

            # Store in replay buffer
            agent.replay_buffer.add(state, action, shaped_reward, next_state, done)
            state = next_state

            # Train
            if len(agent.replay_buffer) >= 32:
                states, actions, rewards, next_states, dones = agent.replay_buffer.sample(32)
                agent.train_step(states, actions, rewards, next_states, dones)

            # Update target model
            if frame_count % update_target_every == 0:
                agent.update_target_model()

            if done:
                break

        # Episode-based epsilon decay
        if frame_count >= agent.epsilon_random_frames:
            decay_per_episode = (agent.epsilon_max - agent.epsilon_min) / (max_episodes * 0.7)  # Decay over first 70% of episodes
            agent.epsilon = max(agent.epsilon_min, agent.epsilon - decay_per_episode)

        episode_rewards.append(episode_reward)
        avg_reward = np.mean(episode_rewards[-log_every:])

        # Log both original and shaped rewards
        with open(log_file_path, "a") as f:
            f.write(f"{episode + 1},{avg_reward:.2f},{agent.epsilon:.4f},{episode_original_reward:.2f},{episode_reward:.2f},{frame_count}\n")


        # Save model conditionally
        if (episode + 1) % 100 == 0 or avg_reward > best_avg_reward:
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
                print(f"New best average reward: {best_avg_reward:.2f}. Saving best model.")
            agent.model.save(f"{model_dir}/episode{episode + 1}_space_invaders_model.keras")
            print(f"Episode {episode + 1}, Avg Reward: {avg_reward:.2f}, Original Reward: {episode_original_reward:.2f}, "
              f"Shaped Reward: {episode_reward:.2f}, Epsilon: {agent.epsilon:.4f}, Frame Count: {frame_count} ")

    # Save final model
    agent.model.save(f"{model_dir}/final_space_invaders_model.keras")
    print(f"Episode {episode + 1}, Avg Reward: {avg_reward:.2f}, Original Reward: {episode_original_reward:.2f}, "
              f"Shaped Reward: {episode_reward:.2f}, Epsilon: {agent.epsilon:.4f}, Frame Count: {frame_count} ")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# File path to your log file
filepath = '/content/drive/MyDrive/SpaceInvaders/logs_2/training_log_2.txt'

# Read the data
df = pd.read_csv(filepath, sep=',', index_col=False)

# Directory to save the plot
save_path = '/content/drive/MyDrive/SpaceInvaders/plots_2/training_log_plot_2.png'

# Create a figure with two y-axes
fig, ax1 = plt.subplots(figsize=(15, 7))

# Plot Average Reward on the primary y-axis
color1 = 'blue'
ax1.set_xlabel('Episode')
ax1.set_ylabel('Average Reward', color=color1)
ax1.plot(df['Episode'], df['Avg_Reward'], color=color1, alpha=0.7, label='Average Reward')
ax1.tick_params(axis='y', labelcolor=color1)

# Create a secondary y-axis for Epsilon
ax2 = ax1.twinx()
color2 = 'red'
ax2.set_ylabel('Epsilon', color=color2)
ax2.plot(df['Episode'], df['Epsilon'], color=color2, alpha=0.7, label='Epsilon')
ax2.tick_params(axis='y', labelcolor=color2)

# Title and layout
plt.title('Reinforcement Learning Training Log 2: Reward and Exploration Rate', fontsize=16)
fig.tight_layout()

# Add a legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Grid for better readability
ax1.grid(True, linestyle='--', alpha=0.7)

# Save the plot
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"Plot saved at: {save_path}")

# Optional: Calculate and print some statistics
print(f"Total Episodes: {len(df)}")
print(f"\nReward Statistics:")
print(f"Minimum Reward: {df['Avg_Reward'].min():.2f}")
print(f"Maximum Reward: {df['Avg_Reward'].max():.2f}")
print(f"Mean Reward: {df['Avg_Reward'].mean():.2f}")
print(f"Median Reward: {df['Avg_Reward'].median():.2f}")

print(f"\nEpsilon Statistics:")
print(f"Initial Epsilon: {df['Epsilon'].iloc[0]:.4f}")
print(f"Final Epsilon: {df['Epsilon'].iloc[-1]:.4f}")

# Show the plot (optional, depends on your environment)
plt.show()

## Outcome Try 2

Looking at both sets of results, here's a comparative analysis:

1. Reward Stability:
- Try 2 shows a more consistent baseline performance, with fewer extreme negative rewards (minimum 2.45 vs -27.55)
- The maximum reward decreased (834.24 vs 1003.80), but this might actually be positive as it suggests less erratic behavior
- Mean rewards are very similar (206.75 vs 207.78), indicating comparable overall performance
- Median rewards are also close (178.39 vs 179.55), suggesting stable learning

2. Learning Progression:
- Try 2's epsilon decay over 70% of episodes (vs 30%) shows a more gradual exploration reduction
- The lower final epsilon (0.01 vs 0.1) allowed for more exploitation of learned strategies
- The reward variance appears more uniform across episodes in Try 2

3. Key Improvements:
- Elimination of severe negative rewards
- More consistent performance across episodes
- Better balance between exploration and exploitation

4. Areas Still Needing Attention:
- High variance still persists, though more uniformly distributed
- No clear upward trend in average reward over time
- Occasional performance spikes suggest potential for better consistency

To further improve, we might consider:
1. Implementing a more sophisticated reward normalization strategy
2. Adding experience prioritization in the replay buffer
3. Adjusting the network architecture or learning rate schedule

## Try 3

In this try we focused on

    -  A more sophisticated reward strategy for shooting reward
    - Strategic Gameplay Rewards:
        Column clearing bonus (+0.2)
        Smart shield usage (+0.05) with damage penalty (-0.1)
        Bullet dodging reward (+0.1)
        Wave survival bonus (+0.3)
        Speed efficiency bonus (+0.2)
        The Overall Shaping Strategy:
         Maintains the core mechanics from try 2 (movement, basic shooting)
        Adds strategic depth with situational bonuses
        Introduces penalties for inefficient play
        Creates a more complete reward hierarchy

In [ ]:

import tensorflow as tf
from keras import layers
import gymnasium as gym
from gymnasium.wrappers import FrameStackObservation, AtariPreprocessing
import numpy as np
import os
import ale_py
import matplotlib as plt

class ReplayBuffer:
    def __init__(self, max_size, state_shape, num_actions):
        self.max_size = max_size
        self.states = np.zeros((max_size, *state_shape), dtype=np.float32)
        self.actions = np.zeros(max_size, dtype=np.int32)
        self.rewards = np.zeros(max_size, dtype=np.float32)
        self.next_states = np.zeros((max_size, *state_shape), dtype=np.float32)
        self.dones = np.zeros(max_size, dtype=np.float32)
        self.index = 0
        self.is_full = False

    def add(self, state, action, reward, next_state, done):
        idx = self.index % self.max_size
        self.states[idx] = state
        self.actions[idx] = action
        self.rewards[idx] = reward
        self.next_states[idx] = next_state
        self.dones[idx] = done
        self.index += 1
        if self.index >= self.max_size:
            self.is_full = True

    def sample(self, batch_size):
        max_index = self.max_size if self.is_full else self.index
        indices = np.random.choice(max_index, batch_size, replace=False)
        return (
            self.states[indices],
            self.actions[indices],
            self.rewards[indices],
            self.next_states[indices],
            self.dones[indices]
        )

    def __len__(self):
        return self.max_size if self.is_full else self.index

class RewardShaper:
    def __init__(self):
        self.previous_score = 0
        self.previous_lives = 3
        self.steps_since_last_shot = 0
        self.last_known_position = None
        self.consecutive_no_movement = 0

    def calculate_shaped_reward(self, info, reward, action, position):
      shaped_reward = reward  # Start with original reward

      # Track score improvements - reduced multiplier
      current_score = info.get('score', 0)
      if current_score > self.previous_score:
          score_improvement = current_score - self.previous_score
          shaped_reward += 0.05 * score_improvement  # Reduced from 0.1 to 0.05
          self.previous_score = current_score

      # Penalize death - made less punitive
      current_lives = info.get('lives', 3)
      if current_lives < self.previous_lives:
          shaped_reward -= 0.5  # Reduced from 1.0 to 0.5
          self.previous_lives = current_lives

      # Movement-based shaping - more lenient thresholds
      if self.last_known_position is not None:
          movement = abs(position - self.last_known_position)
          if movement < 0.005:  # Reduced threshold from 0.01 to 0.005
              self.consecutive_no_movement += 1
              if self.consecutive_no_movement > 15:  # Increased from 10 to 15
                  shaped_reward -= 0.05  # Reduced penalty from 0.1 to 0.05
          else:
              self.consecutive_no_movement = 0
              # Smaller movement reward
              shaped_reward += 0.005  # Reduced from 0.01 to 0.005

      self.last_known_position = position

      # Shooting behavior shaping - more balanced
      if action == 1:  # Shoot action
          self.steps_since_last_shot = 0
          if info.get('hit', False):  # Reward successful hits
              shaped_reward += 0.1
              if info.get('target_is_back_row', False):
                  shaped_reward += 0.2  # Additional bonus for back-row targets
              if info.get('target_is_ufo', False):
                  shaped_reward += 0.2  # Additional bonus for UFO hits
          else:
              shaped_reward -= 0.05  # Penalize missed shots
      else:
          self.steps_since_last_shot += 1
          if self.steps_since_last_shot > 30:  # Increased from 20 to 30
              shaped_reward -= 0.02  # Reduced from 0.05 to 0.02

      # Column clearing incentive
      if info.get('cleared_column', False):
          shaped_reward += 0.2  # Reward for clearing a column

      # Shield usage shaping
      if info.get('near_shield', False) and not info.get('destroying_shield', False):
          shaped_reward += 0.05  # Incentive for smart shield use
      if info.get('destroying_shield', False):
          shaped_reward -= 0.1  # Penalty for damaging own shield

      # Dodge enemy bullets
      if info.get('dodged_bullet', False):
          shaped_reward += 0.1  # Bonus for successful evasion

      # Survival bonus for completing waves
      if info.get('survived_wave', False):
          shaped_reward += 0.3  # Bonus for wave survival

      # Efficiency bonus for faster wave clear
      if info.get('cleared_wave_faster_than_average', False):
          shaped_reward += 0.2  # Efficiency reward

      # Ensure reward doesn't go too negative
      return max(-0.5, shaped_reward)  # Add a floor to prevent extreme negative rewards



class DQNAgent:
    def __init__(self, state_shape, num_actions, learning_rate=0.00025):
        self.state_shape = state_shape
        self.num_actions = num_actions
        self.model = self.create_q_model(state_shape, num_actions)
        self.target_model = self.create_q_model(state_shape, num_actions)
        self.target_model.set_weights(self.model.get_weights())
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0)
        self.replay_buffer = ReplayBuffer(max_size=100000, state_shape=state_shape, num_actions=num_actions)
        self.gamma = 0.99
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_max = 1.0
        self.epsilon_random_frames = 50000  # Initial random frames
        self.epsilon_greedy_frames = 500000.0  # Total frames over which to decay epsilon

    def create_q_model(self, input_shape, num_actions):
        return tf.keras.Sequential([
            layers.Input(shape=input_shape),
            layers.Conv2D(32, kernel_size=8, strides=4, activation="relu", kernel_initializer='he_uniform'),
            layers.Conv2D(64, kernel_size=4, strides=2, activation="relu", kernel_initializer='he_uniform'),
            layers.Conv2D(64, kernel_size=3, strides=1, activation="relu", kernel_initializer='he_uniform'),
            layers.Flatten(),
            layers.Dense(512, activation="relu", kernel_initializer='he_uniform'),
            layers.Dense(num_actions, activation="linear")
        ])

    def get_action(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(self.num_actions)
        state_tensor = tf.convert_to_tensor(state[np.newaxis, ...], dtype=tf.float32)
        q_values = self.model(state_tensor, training=False)
        return tf.argmax(q_values[0]).numpy()

    @tf.function
    def train_step(self, states, actions, rewards, next_states, dones):
        future_rewards = self.target_model(next_states, training=False)
        target_q_values = rewards + self.gamma * tf.reduce_max(future_rewards, axis=1) * (1 - dones)
        with tf.GradientTape() as tape:
            q_values = self.model(states, training=True)
            q_action = tf.reduce_sum(tf.one_hot(actions, self.num_actions) * q_values, axis=1)
            loss = tf.keras.losses.Huber()(target_q_values, q_action)
        grads = tape.gradient(loss, self.model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.model.trainable_variables))
        return loss

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

def get_agent_position(state):
    """Extract the agent's position from the state"""
    bottom_slice = state[-10:, :, -1]  # Look at the bottom portion of the last frame
    agent_x = np.mean(np.where(bottom_slice > 0.5)[1]) if np.any(bottom_slice > 0.5) else None
    if agent_x is not None:
        return agent_x / state.shape[1]  # Normalize position
    return 0.5  # Return center if agent not found

def main():
    # Paths for saving
    model_dir = "/content/drive/MyDrive/SpaceInvaders/models_3"
    plot_dir = "/content/drive/MyDrive/SpaceInvaders/plots_3"
    log_dir = "/content/drive/MyDrive/SpaceInvaders/logs_3"
    log_file_path = os.path.join(log_dir, "training_log_3.txt")

    # Create all necessary directories
    for directory in [model_dir, plot_dir, log_dir]:
        os.makedirs(directory, exist_ok=True)

    # Environment setup
    gym.register_envs(ale_py)
    env = gym.make("SpaceInvadersNoFrameskip-v4", render_mode="rgb_array")
    env = AtariPreprocessing(env, frame_skip=4, grayscale_obs=True, scale_obs=True)
    env = FrameStackObservation(env, stack_size=4)

    # Agent and reward shaper setup
    state_shape = (84, 84, 4)
    num_actions = env.action_space.n
    agent = DQNAgent(state_shape, num_actions)
    reward_shaper = RewardShaper()

    # Training parameters
    max_episodes = 1000  # Reduced from 2000
    max_steps_per_episode = 3000
    update_target_every = 1000
    log_every = 1
    episode_rewards = []
    frame_count = 0
    best_avg_reward = -float('inf')

    with open(log_file_path, "w") as f:
        f.write("Episode,Avg_Reward,Epsilon,Original_Reward,Shaped_Reward\n")

    # Training loop
    for episode in range(max_episodes):
        state, info = env.reset()
        state = np.array(state, dtype=np.float32) / 255.0
        state = np.transpose(state, (1, 2, 0))
        episode_reward = 0
        episode_original_reward = 0
        episode_steps = 0

        for step in range(max_steps_per_episode):
            frame_count += 1
            episode_steps += 1

            # Choose action
            if frame_count < agent.epsilon_random_frames:
                action = np.random.randint(num_actions)
            else:
                action = agent.get_action(state)

            # Step environment
            next_state, reward, done, _, info = env.step(action)
            next_state = np.array(next_state, dtype=np.float32) / 255.0
            next_state = np.transpose(next_state, (1, 2, 0))

            # Shape the reward
            agent_position = get_agent_position(state)
            shaped_reward = reward_shaper.calculate_shaped_reward(info, reward, action, agent_position)

            # Store original reward for logging
            episode_original_reward += reward
            episode_reward += shaped_reward

            # Store in replay buffer
            agent.replay_buffer.add(state, action, shaped_reward, next_state, done)
            state = next_state

            # Train
            if len(agent.replay_buffer) >= 32:
                states, actions, rewards, next_states, dones = agent.replay_buffer.sample(32)
                agent.train_step(states, actions, rewards, next_states, dones)

            # Update target model
            if frame_count % update_target_every == 0:
                agent.update_target_model()

            if done:
                break

        # Episode-based epsilon decay
        if frame_count >= agent.epsilon_random_frames:
            decay_per_episode = (agent.epsilon_max - agent.epsilon_min) / (max_episodes * 0.7)  # Decay over first 70% of episodes
            agent.epsilon = max(agent.epsilon_min, agent.epsilon - decay_per_episode)

        episode_rewards.append(episode_reward)
        avg_reward = np.mean(episode_rewards[-log_every:])

        # Log both original and shaped rewards
        with open(log_file_path, "a") as f:
            f.write(f"{episode + 1},{avg_reward:.2f},{agent.epsilon:.4f},{episode_original_reward:.2f},{episode_reward:.2f},{frame_count}\n")


        # Save model conditionally
        if (episode + 1) % 100 == 0 or avg_reward > best_avg_reward:
            if avg_reward > best_avg_reward:
                best_avg_reward = avg_reward
                print(f"New best average reward: {best_avg_reward:.2f}. Saving best model.")
            agent.model.save(f"{model_dir}/episode{episode + 1}_space_invaders_model.keras")
            print(f"Episode {episode + 1}, Avg Reward: {avg_reward:.2f}, Original Reward: {episode_original_reward:.2f}, "
              f"Shaped Reward: {episode_reward:.2f}, Epsilon: {agent.epsilon:.4f}, Frame Count: {frame_count} ")

    # Save final model
    agent.model.save(f"{model_dir}/final_space_invaders_model.keras")
    print(f"Episode {episode + 1}, Avg Reward: {avg_reward:.2f}, Original Reward: {episode_original_reward:.2f}, "
              f"Shaped Reward: {episode_reward:.2f}, Epsilon: {agent.epsilon:.4f}, Frame Count: {frame_count} ")

if __name__ == "__main__":
    main()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# File path to your log file
filepath = '/content/drive/MyDrive/SpaceInvaders/logs_3/training_log_3.txt'

# Read the data
df = pd.read_csv(filepath, sep=',', index_col=False)

# Directory to save the plot
save_path = '/content/drive/MyDrive/SpaceInvaders/plots_3/training_log_plot_3.png'

# Create a figure with two y-axes
fig, ax1 = plt.subplots(figsize=(15, 7))

# Plot Average Reward on the primary y-axis
color1 = 'blue'
ax1.set_xlabel('Episode')
ax1.set_ylabel('Average Reward', color=color1)
ax1.plot(df['Episode'], df['Avg_Reward'], color=color1, alpha=0.7, label='Average Reward')
ax1.tick_params(axis='y', labelcolor=color1)

# Create a secondary y-axis for Epsilon
ax2 = ax1.twinx()
color2 = 'red'
ax2.set_ylabel('Epsilon', color=color2)
ax2.plot(df['Episode'], df['Epsilon'], color=color2, alpha=0.7, label='Epsilon')
ax2.tick_params(axis='y', labelcolor=color2)

# Title and layout
plt.title('Reinforcement Learning Training Log 3: Reward and Exploration Rate', fontsize=16)
fig.tight_layout()

# Add a legend
lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

# Grid for better readability
ax1.grid(True, linestyle='--', alpha=0.7)

# Save the plot
plt.savefig(save_path, dpi=300, bbox_inches='tight')
print(f"Plot saved at: {save_path}")

# Optional: Calculate and print some statistics
print(f"Total Episodes: {len(df)}")
print(f"\nReward Statistics:")
print(f"Minimum Reward: {df['Avg_Reward'].min():.2f}")
print(f"Maximum Reward: {df['Avg_Reward'].max():.2f}")
print(f"Mean Reward: {df['Avg_Reward'].mean():.2f}")
print(f"Median Reward: {df['Avg_Reward'].median():.2f}")

print(f"\nEpsilon Statistics:")
print(f"Initial Epsilon: {df['Epsilon'].iloc[0]:.4f}")
print(f"Final Epsilon: {df['Epsilon'].iloc[-1]:.4f}")

# Show the plot
plt.show()

## Outcome Try 3


Try 2:              
Min: 2.45            
Max: 834.24          
Mean: 206.75         
Median: 178.39       


  Try 3:
  Min: -17.60                         
  Max: 631.96                     
  Mean: 150.79                    
  Median: 126.97            

#### Key Observations:

Overall lower performance (mean dropped by ~27%)

Return of negative rewards (-17.60)

Lower maximum rewards (drop of ~200 points)

More compressed reward range

Similar variance pattern but at a lower baseline


Likely Issues with New Reward Structure:

The complex reward system may be creating competing objectives

Larger reward values (0.1-0.3) might be overwhelming the base game rewards

Multiple simultaneous rewards could be creating noise in the learning signal

The agent might be struggling to correlate delayed rewards with actions

#### Suggested Improvements if there were time:


Scale back reward complexity

Normalize reward values to be more consistent

Focus on fewer, clearer reward signals

Consider reducing the magnitude gap between different rewards

# Space Invaders DQN Training Analysis - 3 Rounds

## Performance Metrics Comparison

| Metric | Try 1 | Try 2 | Try 3 |
|--------|-------|-------|-------|
| Minimum Reward | -27.55 | 2.45 | -17.60 |
| Maximum Reward | 1003.80 | 834.24 | 631.96 |
| Mean Reward | 207.78 | 206.75 | 150.79 |
| Median Reward | 179.55 | 178.39 | 126.97 |
| Final Epsilon | 0.1000 | 0.0100 | 0.0100 |

## Key Findings Per Round

### Try 1 (Basic Implementation)
- Highest maximum reward but also largest negative rewards
- High variance in performance
- Conservative exploration (epsilon = 0.1)
- Showed potential for high scores but lacked consistency

### Try 2 (Balanced Approach)
- Eliminated negative rewards
- More stable performance
- Better exploration-exploitation balance (epsilon = 0.01)
- Most consistent mean/median performance
- Best overall balance of stability and performance

### Try 3 (Complex Reward Shaping)
- Added sophisticated reward mechanisms
- Lower overall performance
- Reintroduction of negative rewards
- More compressed reward range
- Complex reward structure may have hindered learning

## Learning Trends
1. Epsilon decay became more gradual across attempts
2. Performance consistency improved from Try 1 to Try 2
3. Added complexity in Try 3 led to decreased performance
4. Try 2 achieved the best balance of exploration and stability

## Final Verdict
Try 2 emerged as the most successful implementation, suggesting that moderate reward shaping with balanced exploration-exploitation tradeoff produces better results than either basic or complex approaches.

In [ ]:
import tensorflow as tf
import gymnasium as gym
from gymnasium.wrappers import FrameStackObservation, AtariPreprocessing
import numpy as np
import ale_py
import time
import cv2

def load_model(model_path):
    """Load a saved model"""
    return tf.keras.models.load_model(model_path)

def get_agent_position(state):
    """Extract the agent's position from the state"""
    bottom_slice = state[-10:, :, -1]
    agent_x = np.mean(np.where(bottom_slice > 0.5)[1]) if np.any(bottom_slice > 0.5) else None
    if agent_x is not None:
        return agent_x / state.shape[1]
    return 0.5

def evaluate_model(model_path, num_episodes=5, render=True):
    """Evaluate a model's performance"""
    # Environment setup
    env = gym.make("SpaceInvadersNoFrameskip-v4", render_mode="rgb_array")
    env = AtariPreprocessing(env, frame_skip=4, grayscale_obs=True, scale_obs=True)
    env = FrameStackObservation(env, stack_size=4)
    
    model = load_model(model_path)
    episode_rewards = []

    for episode in range(num_episodes):
        state, info = env.reset()
        state = np.array(state, dtype=np.float32) / 255.0
        state = np.transpose(state, (1, 2, 0))
        episode_reward = 0
        done = False
        steps = 0

        while not done and steps < 3000:  # Same step limit as training
            # Get action from model
            state_tensor = tf.convert_to_tensor(state[np.newaxis, ...], dtype=tf.float32)
            q_values = model(state_tensor, training=False)
            action = tf.argmax(q_values[0]).numpy()

            # Take action
            next_state, reward, done, _, info = env.step(action)
            next_state = np.array(next_state, dtype=np.float32) / 255.0
            next_state = np.transpose(next_state, (1, 2, 0))
            
            episode_reward += reward
            state = next_state
            steps += 1

            if render:
                # Get the RGB frame from the environment
                frame = env.render()
                
                # Display the frame using cv2
                cv2.imshow('Space Invaders', cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))
                cv2.waitKey(20)  # 20ms delay between frames

        episode_rewards.append(episode_reward)
        print(f"Episode {episode + 1} Reward: {episode_reward}")

    if render:
        cv2.destroyAllWindows()

    return np.mean(episode_rewards)

def main():
    # Paths to your saved models
    model_paths = [
        "/models/models_1/final_space_invaders_model_1.keras",
        "/models/models_2/final_space_invaders_model_2.keras",
        "/models/models_3/final_space_invaders_model_3.keras"
    ]

    print("Evaluating all models...")
    for i, model_path in enumerate(model_paths, 1):
        print(f"\nEvaluating Model {i}")
        try:
            avg_reward = evaluate_model(model_path, num_episodes=5, render=True)
            print(f"Model {i} Average Reward over 5 episodes: {avg_reward}")
            time.sleep(2)  # Pause between models
        except Exception as e:
            print(f"Error evaluating model {i}: {str(e)}")

if __name__ == "__main__":
    main()